In [46]:
from sklearn.preprocessing import StandardScaler
import threadpoolctl as tpc
import sklearn 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd

In [47]:
print(tpc.__version__)

3.1.0


In [48]:
file = pd.read_csv('.\Resources\games.csv')
file.head()

,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,...,Rank:partygames,Rank:childrensgames,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens
0,1,Die Macher,die macher game seven sequential political rac...,1986,4.3206,7.61428,7.10363,1.57979,3,5,...,21926,21926,0,1,0,0,0,0,0,0
1,2,Dragonmaster,dragonmaster tricktaking card game base old ga...,1981,1.9630,6.64537,5.78447,1.45440,3,4,...,21926,21926,0,1,0,0,0,0,0,0
2,3,Samurai,samurai set medieval japan player compete gain...,1998,2.4859,7.45601,7.23994,1.18227,2,4,...,21926,21926,0,1,0,0,0,0,0,0
3,4,Tal der Könige,triangular box luxurious large block tal der k...,1992,2.6667,6.60006,5.67954,1.23129,2,4,...,21926,21926,0,0,0,0,0,0,0,0
4,5,Acquire,acquire player strategically invest business t...,1964,2.5031,7.33861,7.14189,1.33583,2,6,...,21926,21926,0,1,0,0,0,0,0,0


In [49]:
file.columns

Index(['BGGId', 'Name', 'Description', 'YearPublished', 'GameWeight',
       'AvgRating', 'BayesAvgRating', 'StdDev', 'MinPlayers', 'MaxPlayers',
       'ComAgeRec', 'LanguageEase', 'BestPlayers', 'GoodPlayers', 'NumOwned',
       'NumWant', 'NumWish', 'NumWeightVotes', 'MfgPlaytime', 'ComMinPlaytime',
       'ComMaxPlaytime', 'MfgAgeRec', 'NumUserRatings', 'NumComments',
       'NumAlternates', 'NumExpansions', 'NumImplementations',
       'IsReimplementation', 'Family', 'Kickstarted', 'ImagePath',
       'Rank:boardgame', 'Rank:strategygames', 'Rank:abstracts',
       'Rank:familygames', 'Rank:thematic', 'Rank:cgs', 'Rank:wargames',
       'Rank:partygames', 'Rank:childrensgames', 'Cat:Thematic',
       'Cat:Strategy', 'Cat:War', 'Cat:Family', 'Cat:CGS', 'Cat:Abstract',
       'Cat:Party', 'Cat:Childrens'],
      dtype='object')

In [50]:
file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21925 entries, 0 to 21924
Data columns (total 48 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   BGGId                21925 non-null  int64  
 1   Name                 21925 non-null  object 
 2   Description          21924 non-null  object 
 3   YearPublished        21925 non-null  int64  
 4   GameWeight           21925 non-null  float64
 5   AvgRating            21925 non-null  float64
 6   BayesAvgRating       21925 non-null  float64
 7   StdDev               21925 non-null  float64
 8   MinPlayers           21925 non-null  int64  
 9   MaxPlayers           21925 non-null  int64  
 10  ComAgeRec            16395 non-null  float64
 11  LanguageEase         16034 non-null  float64
 12  BestPlayers          21925 non-null  int64  
 13  GoodPlayers          21925 non-null  object 
 14  NumOwned             21925 non-null  int64  
 15  NumWant              21925 non-null 

In [51]:
# Removing from original dataset the columns related to players only, image urls and columns with null values
# In addition, based on the percentage of populated columns (values > 0) in the dataset (see Data Cleaning.ipynb file) ,
# we propose to remove the columns NumComments based on lack of data. 
# NumAlternates, NumExpansions, and NumImplementations also have low percentage of games populated, but they carry 
# value as they indicate the number of alternate versions, expansions, and implementations the games have. 

nn_game_df = pd.DataFrame(file, columns=['BGGId', 'Name', 'YearPublished', 'GameWeight',
       'AvgRating', 'BayesAvgRating', 'StdDev', 'MinPlayers', 'MaxPlayers',
       'NumOwned','NumWant', 'NumWish', 'MfgPlaytime', 'ComMinPlaytime',
       'ComMaxPlaytime', 'MfgAgeRec', 'NumUserRatings',
       'NumAlternates', 'NumExpansions', 'NumImplementations',
       'IsReimplementation', 'Kickstarted',
       'Rank:boardgame', 'Rank:strategygames', 'Rank:abstracts',
       'Rank:familygames', 'Rank:thematic', 'Rank:cgs', 'Rank:wargames',
       'Rank:partygames', 'Rank:childrensgames', 'Cat:Thematic',
       'Cat:Strategy', 'Cat:War', 'Cat:Family', 'Cat:CGS', 'Cat:Abstract',
       'Cat:Party', 'Cat:Childrens'])

nn_game_df.head()

,BGGId,Name,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,NumOwned,...,Rank:partygames,Rank:childrensgames,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens
0,1,Die Macher,1986,4.3206,7.61428,7.10363,1.57979,3,5,7498,...,21926,21926,0,1,0,0,0,0,0,0
1,2,Dragonmaster,1981,1.9630,6.64537,5.78447,1.45440,3,4,1285,...,21926,21926,0,1,0,0,0,0,0,0
2,3,Samurai,1998,2.4859,7.45601,7.23994,1.18227,2,4,15578,...,21926,21926,0,1,0,0,0,0,0,0
3,4,Tal der Könige,1992,2.6667,6.60006,5.67954,1.23129,2,4,638,...,21926,21926,0,0,0,0,0,0,0,0
4,5,Acquire,1964,2.5031,7.33861,7.14189,1.33583,2,6,23735,...,21926,21926,0,1,0,0,0,0,0,0


In [52]:
nn_game_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21925 entries, 0 to 21924
Data columns (total 39 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   BGGId                21925 non-null  int64  
 1   Name                 21925 non-null  object 
 2   YearPublished        21925 non-null  int64  
 3   GameWeight           21925 non-null  float64
 4   AvgRating            21925 non-null  float64
 5   BayesAvgRating       21925 non-null  float64
 6   StdDev               21925 non-null  float64
 7   MinPlayers           21925 non-null  int64  
 8   MaxPlayers           21925 non-null  int64  
 9   NumOwned             21925 non-null  int64  
 10  NumWant              21925 non-null  int64  
 11  NumWish              21925 non-null  int64  
 12  MfgPlaytime          21925 non-null  int64  
 13  ComMinPlaytime       21925 non-null  int64  
 14  ComMaxPlaytime       21925 non-null  int64  
 15  MfgAgeRec            21925 non-null 

In [53]:
nn_df_scaled = StandardScaler().fit_transform(nn_game_df[[
                                                        'YearPublished', 'GameWeight',
                   'AvgRating', 'BayesAvgRating', 'StdDev', 'MinPlayers', 'MaxPlayers',
                   'NumOwned','NumWant', 'NumWish', 'MfgPlaytime', 'ComMinPlaytime',
                   'ComMaxPlaytime', 'MfgAgeRec', 'NumUserRatings',
                   'NumAlternates', 'NumExpansions', 'NumImplementations',
                   'IsReimplementation', 'Kickstarted',
                   'Rank:boardgame', 'Rank:strategygames', 'Rank:abstracts',
                   'Rank:familygames', 'Rank:thematic', 'Rank:cgs', 'Rank:wargames',
                   'Rank:partygames', 'Rank:childrensgames', 'Cat:Thematic',
                   'Cat:Strategy', 'Cat:War', 'Cat:Family', 'Cat:CGS', 'Cat:Abstract',
                   'Cat:Party', 'Cat:Childrens']])


nn_df_transformed = pd.DataFrame(nn_df_scaled, columns=[
                                                        'YearPublished', 'GameWeight',
                   'AvgRating', 'BayesAvgRating', 'StdDev', 'MinPlayers', 'MaxPlayers',
                   'NumOwned','NumWant', 'NumWish', 'MfgPlaytime', 'ComMinPlaytime',
                   'ComMaxPlaytime', 'MfgAgeRec', 'NumUserRatings',
                   'NumAlternates', 'NumExpansions', 'NumImplementations',
                   'IsReimplementation', 'Kickstarted',
                   'Rank:boardgame', 'Rank:strategygames', 'Rank:abstracts',
                   'Rank:familygames', 'Rank:thematic', 'Rank:cgs', 'Rank:wargames',
                   'Rank:partygames', 'Rank:childrensgames', 'Cat:Thematic',
                   'Cat:Strategy', 'Cat:War', 'Cat:Family', 'Cat:CGS', 'Cat:Abstract',
                   'Cat:Party', 'Cat:Childrens'])
nn_df_transformed

,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,NumOwned,NumWant,NumWish,...,Rank:partygames,Rank:childrensgames,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens
0,0.002377,2.754499,1.275510,3.881596,0.222067,1.432245,-0.047146,1.139054,3.917263,2.296305,...,0.173395,0.204594,-0.243162,2.907663,-0.438064,-0.34367,-0.118379,-0.231474,-0.173402,-0.204609
1,-0.021154,-0.022535,0.236416,0.270454,-0.217018,1.432245,-0.113749,-0.034539,0.258494,-0.047507,...,0.173395,0.204594,-0.243162,2.907663,-0.438064,-0.34367,-0.118379,-0.231474,-0.173402,-0.204609
2,0.058853,0.593393,1.105776,4.254739,-1.169949,-0.010595,-0.113749,2.665310,6.458785,4.085872,...,0.173395,0.204594,-0.243162,2.907663,-0.438064,-0.34367,-0.118379,-0.231474,-0.173402,-0.204609
3,0.030615,0.806359,0.187824,-0.016787,-0.998293,-0.010595,-0.113749,-0.156753,0.104979,-0.133751,...,0.173395,0.204594,-0.243162,-0.343919,-0.438064,-0.34367,-0.118379,-0.231474,-0.173402,-0.204609
4,-0.101161,0.613653,0.979872,3.986331,-0.632220,-0.010595,0.019457,4.206111,4.318107,3.097869,...,0.173395,0.204594,-0.243162,2.907663,-0.438064,-0.34367,-0.118379,-0.231474,-0.173402,-0.204609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21920,0.167097,-2.334766,1.099331,-0.440874,0.135013,1.432245,-0.113749,-0.268577,-0.270279,-0.246629,...,0.173395,0.204594,-0.243162,-0.343919,-0.438064,-0.34367,-0.118379,-0.231474,-0.173402,-0.204609
21921,0.157685,-0.371548,1.606336,-0.335181,-2.178684,-1.453435,-0.246956,-0.248555,-0.219108,-0.223800,...,0.173395,0.204594,-0.243162,-0.343919,-0.438064,-0.34367,-0.118379,-0.231474,-0.173402,-0.204609
21922,0.167097,3.039435,0.584239,-0.399758,8.881123,-1.453435,-0.180353,-0.276133,-0.338508,-0.273263,...,0.173395,0.204594,-0.243162,-0.343919,-0.438064,-0.34367,-0.118379,-0.231474,-0.173402,-0.204609
21923,0.167097,-1.156859,0.061587,-0.436248,-1.247688,-1.453435,0.419076,-0.256488,-0.270279,-0.227605,...,0.173395,0.204594,-0.243162,-0.343919,-0.438064,-0.34367,-0.118379,-0.231474,-0.173402,-0.204609


In [54]:
nn_df_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21925 entries, 0 to 21924
Data columns (total 37 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   YearPublished        21925 non-null  float64
 1   GameWeight           21925 non-null  float64
 2   AvgRating            21925 non-null  float64
 3   BayesAvgRating       21925 non-null  float64
 4   StdDev               21925 non-null  float64
 5   MinPlayers           21925 non-null  float64
 6   MaxPlayers           21925 non-null  float64
 7   NumOwned             21925 non-null  float64
 8   NumWant              21925 non-null  float64
 9   NumWish              21925 non-null  float64
 10  MfgPlaytime          21925 non-null  float64
 11  ComMinPlaytime       21925 non-null  float64
 12  ComMaxPlaytime       21925 non-null  float64
 13  MfgAgeRec            21925 non-null  float64
 14  NumUserRatings       21925 non-null  float64
 15  NumAlternates        21925 non-null 

In [79]:
# Creating csv file with scaled data to be used by the app.py
name_id_df = pd.DataFrame(file, columns=['BGGId', 'Name'])
file1 = pd.concat([name_id_df,nn_df_transformed], axis=1)
file1

,BGGId,Name,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,NumOwned,...,Rank:partygames,Rank:childrensgames,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens
0,1,Die Macher,0.002377,2.754499,1.275510,3.881596,0.222067,1.432245,-0.047146,1.139054,...,0.173395,0.204594,-0.243162,2.907663,-0.438064,-0.34367,-0.118379,-0.231474,-0.173402,-0.204609
1,2,Dragonmaster,-0.021154,-0.022535,0.236416,0.270454,-0.217018,1.432245,-0.113749,-0.034539,...,0.173395,0.204594,-0.243162,2.907663,-0.438064,-0.34367,-0.118379,-0.231474,-0.173402,-0.204609
2,3,Samurai,0.058853,0.593393,1.105776,4.254739,-1.169949,-0.010595,-0.113749,2.665310,...,0.173395,0.204594,-0.243162,2.907663,-0.438064,-0.34367,-0.118379,-0.231474,-0.173402,-0.204609
3,4,Tal der Könige,0.030615,0.806359,0.187824,-0.016787,-0.998293,-0.010595,-0.113749,-0.156753,...,0.173395,0.204594,-0.243162,-0.343919,-0.438064,-0.34367,-0.118379,-0.231474,-0.173402,-0.204609
4,5,Acquire,-0.101161,0.613653,0.979872,3.986331,-0.632220,-0.010595,0.019457,4.206111,...,0.173395,0.204594,-0.243162,2.907663,-0.438064,-0.34367,-0.118379,-0.231474,-0.173402,-0.204609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21920,347146,Salvage,0.167097,-2.334766,1.099331,-0.440874,0.135013,1.432245,-0.113749,-0.268577,...,0.173395,0.204594,-0.243162,-0.343919,-0.438064,-0.34367,-0.118379,-0.231474,-0.173402,-0.204609
21921,347521,Blitzkrieg!: World War Two in 20 Minutes,0.157685,-0.371548,1.606336,-0.335181,-2.178684,-1.453435,-0.246956,-0.248555,...,0.173395,0.204594,-0.243162,-0.343919,-0.438064,-0.34367,-0.118379,-0.231474,-0.173402,-0.204609
21922,348955,Rock Paper Scissors: Deluxe Edition,0.167097,3.039435,0.584239,-0.399758,8.881123,-1.453435,-0.180353,-0.276133,...,0.173395,0.204594,-0.243162,-0.343919,-0.438064,-0.34367,-0.118379,-0.231474,-0.173402,-0.204609
21923,349131,Splitter,0.167097,-1.156859,0.061587,-0.436248,-1.247688,-1.453435,0.419076,-0.256488,...,0.173395,0.204594,-0.243162,-0.343919,-0.438064,-0.34367,-0.118379,-0.231474,-0.173402,-0.204609


In [80]:
# Importing file to csv to be used by app.py
file1.to_csv('games_scaled.csv', index=True)

In [55]:
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=6)
neigh.fit(nn_df_transformed.iloc[:,0:36])

NearestNeighbors(n_neighbors=6)

In [98]:
nn_game_df[nn_game_df["Name"].str.contains("Trivial")].head()

,BGGId,Name,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,NumOwned,...,Rank:partygames,Rank:childrensgames,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens
969,1297,Trivial Pursuit: Star Wars Classic Trilogy Col...,1998,1.5816,5.57529,5.49673,1.59220,2,4,3478,...,568,21926,0,0,0,1,0,0,1,0
971,1299,Trivial Pursuit: Genus IV,1996,1.8143,5.53983,5.48312,1.60404,2,6,1903,...,575,21926,0,0,0,1,0,0,1,0
1454,2001,Trivial Pursuit: Family Edition,1993,1.7500,5.49387,5.48114,1.48199,2,36,1944,...,21926,510,0,0,0,0,0,0,0,1
1843,2586,Trivial Pursuit: Game Show,1993,1.0000,5.83742,5.50669,1.89477,3,6,93,...,21926,21926,0,0,0,0,0,0,0,0
1981,2819,Trivial Pursuit: Genus 5,2000,1.8372,5.53995,5.49445,1.57132,2,24,1259,...,21926,21926,0,0,0,0,0,0,0,0


In [100]:
game = nn_df_transformed.iloc[969][0:36]
game

YearPublished          0.058853
GameWeight            -0.471788
AvgRating             -0.911177
BayesAvgRating        -0.517222
StdDev                 0.265523
MinPlayers            -0.010595
MaxPlayers            -0.113749
NumOwned               0.379704
NumWant               -0.312922
NumWish               -0.249166
MfgPlaytime           -0.000970
ComMinPlaytime         0.059295
ComMaxPlaytime        -0.000970
MfgAgeRec              0.655391
NumUserRatings         0.171586
NumAlternates          0.041190
NumExpansions         -0.179210
NumImplementations    -0.364041
IsReimplementation    -0.363589
Kickstarted           -0.425574
Rank:boardgame         1.147142
Rank:strategygames     0.343718
Rank:abstracts         0.231446
Rank:familygames      -2.746270
Rank:thematic          0.243126
Rank:cgs               0.118378
Rank:wargames          0.437398
Rank:partygames       -5.698935
Rank:childrensgames    0.204594
Cat:Thematic          -0.243162
Cat:Strategy          -0.343919
Cat:War 

In [101]:
game.to_numpy()

array([ 5.88526214e-02, -4.71788384e-01, -9.11176870e-01, -5.17221842e-01,
        2.65523263e-01, -1.05950841e-02, -1.13749401e-01,  3.79703796e-01,
       -3.12922362e-01, -2.49165740e-01, -9.69560884e-04,  5.92950054e-02,
       -9.69560884e-04,  6.55390857e-01,  1.71585786e-01,  4.11901879e-02,
       -1.79209632e-01, -3.64040912e-01, -3.63589416e-01, -4.25573691e-01,
        1.14714210e+00,  3.43717649e-01,  2.31445825e-01, -2.74626961e+00,
        2.43126014e-01,  1.18377681e-01,  4.37397846e-01, -5.69893515e+00,
        2.04593631e-01, -2.43161630e-01, -3.43918767e-01, -4.38063891e-01,
        2.90976855e+00, -1.18378650e-01, -2.31473561e-01,  5.76695869e+00])

In [102]:
closest_neighbor = neigh.kneighbors(game.to_numpy().reshape(1, -1))
print(closest_neighbor)
print("--------------------------------------------------------")
print(f"Nearest neighbor: ")
print("--------------------------------------------------------")
# print(f"Distance to nearest neighbor: {closest_neighbor[0]}")
print(nn_game_df.iloc[closest_neighbor[1][0][1],1])
print(nn_game_df.iloc[closest_neighbor[1][0][2],1])
print(nn_game_df.iloc[closest_neighbor[1][0][3],1])
print(nn_game_df.iloc[closest_neighbor[1][0][4],1])
print(nn_game_df.iloc[closest_neighbor[1][0][5],1])

(array([[0.        , 0.92053115, 1.1410853 , 1.16615678, 1.19708905,
        1.58767749]]), array([[  969,  5787, 10644,  2116,  1020,  7266]], dtype=int64))
--------------------------------------------------------
Nearest neighbor: 
--------------------------------------------------------
Trivial Pursuit: 90's Time Capsule Edition
Nuts!
Trivial Pursuit: Millennium Edition
Wit's End
Megastar


C:\Users\Priscila\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [75]:
i = closest_neighbor[1][0]
nn_game_df.iloc[i, :]

,BGGId,Name,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,NumOwned,...,Rank:partygames,Rank:childrensgames,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens
1051,1406,Monopoly,1933,1.6388,4.37412,4.30712,1.86399,2,8,42846,...,21926,21926,0,0,0,1,0,0,0,0
159,181,Risk,1959,2.0792,5.59396,5.46271,1.65616,2,6,49849,...,21926,21926,0,0,1,1,0,0,0,0
967,1294,Clue,1949,1.6563,5.66992,5.56904,1.49321,2,6,32383,...,21926,21926,0,0,0,1,0,0,0,0
1741,2425,Battleship,1931,1.2152,4.66125,4.72300,1.51394,2,2,14292,...,21926,846,0,0,0,0,0,0,0,1
377,438,Scotland Yard,1983,1.9547,6.51192,6.36965,1.27186,3,6,24773,...,21926,21926,0,0,0,1,0,0,0,0
918,1219,Labyrinth,1986,1.3665,6.40615,6.23409,1.26842,2,4,21323,...,21926,21926,0,0,0,1,0,0,0,0


In [61]:
s = nn_game_df.iloc[closest_neighbor[1][0][1],:6]
df = pd.DataFrame(s)
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>278</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>BGGId</th>\n      <td>320</td>\n    </tr>\n    <tr>\n      <th>Name</th>\n      <td>Scrabble</td>\n    </tr>\n    <tr>\n      <th>YearPublished</th>\n      <td>1948</td>\n    </tr>\n    <tr>\n      <th>GameWeight</th>\n      <td>2.0878</td>\n    </tr>\n    <tr>\n      <th>AvgRating</th>\n      <td>6.2753</td>\n    </tr>\n    <tr>\n      <th>BayesAvgRating</th>\n      <td>6.16004</td>\n    </tr>\n  </tbody>\n</table>'